In [25]:
import arcpy
import numpy as np
import pandas as pd

In [15]:
# Import Elevation Raster, for no value drops
elevationRas = arcpy.Raster(r'C:\Users\augus\Documents\ArcGIS\Projects\PyROW\PyROW_CA.gdb\elevation')
elevation_array = arcpy.RasterToNumPyArray(elevationRas, nodata_to_value=-9999).flatten()
drop_indices = np.asarray(np.where(elevation_array == -9999), dtype='int').flatten()
np.save('novalue_indices', drop_indices)

In [16]:
drop_indices = np.load('novalue_indices.npy')

In [17]:
elevationRas = arcpy.Raster(r'C:\Users\augus\Documents\ArcGIS\Projects\PyROW\PyROW_CA.gdb\elevation')
elevation_array = np.delete(arcpy.RasterToNumPyArray(elevationRas, nodata_to_value=0).flatten(), drop_indices)

rows_width = elevationRas.width
columns_height = elevationRas.height

lat = np.zeros((columns_height, rows_width))
long = np.zeros((columns_height, rows_width))

lat_value = lambda y : elevationRas.extent.YMax + ((elevationRas.extent.YMin - elevationRas.extent.YMax) / (columns_height - 1)) * y

for n, row in enumerate(lat):
    row[:] = lat_value(n)

lat = lat.flatten()

long_value = lambda x: elevationRas.extent.XMin + ((elevationRas.extent.XMax - elevationRas.extent.XMin) / (rows_width - 1)) * x

long = long.transpose()
for n, psuedorow in enumerate(long):
    psuedorow[:] = long_value(n)
long = long.transpose().flatten()

farray_latitude = np.delete(lat, drop_indices)
farray_longitude = np.delete(long, drop_indices)
farray_elevation = elevation_array

In [18]:
agencyRas = arcpy.Raster(r'C:\Users\augus\Documents\ArcGIS\Projects\PyROW\PyROW_CA.gdb\agencies')

farray_agency = np.delete(arcpy.RasterToNumPyArray(agencyRas, nodata_to_value=0).flatten(), drop_indices)
agency_dict = {0.0: "No Agency",
 1.0: "Fish and Wildlife Service",
 2.0: "Bureau of Land Management",
 3.0: "National Park Service",
 4.0: "Forest Service",
 5.0: "Department of Defense",
 6.0: "Bureau of Reclamation"}

def agency_value_replace(i):
    return agency_dict[i]

agency_mapper = np.vectorize(agency_value_replace)
farray_agency = agency_mapper(farray_agency)

In [23]:
arrays_for_df = {"LATITUDE": farray_latitude,
                 "LONGITUDE": farray_longitude,
                "ELEVATION": farray_elevation,
                "AGENCY": farray_agency}

df = pd.DataFrame(data=arrays_for_df)
df.head()

df.to_parquet("lat_long_height_agency.parquet", compression='None')

In [24]:
base_df = pd.read_parquet("lat_long_height_agency.parquet")
base_df.head()

,LATITUDE,LONGITUDE,ELEVATION,AGENCY
0,5.162364e+06,-1.370879e+07,824.563354,Forest Service
1,5.162364e+06,-1.370869e+07,830.883362,Forest Service
2,5.162364e+06,-1.370859e+07,877.235413,Forest Service
3,5.162364e+06,-1.370849e+07,899.555420,Forest Service
4,5.162364e+06,-1.370839e+07,860.095459,Forest Service


In [ ]:
# Snow data not used. SNOTEL data not widespread enough for correct use.

In [37]:
time_series_data_prefixes = ["drought", "groundcover", "ppt", "tmp", "fire"]

groundcover_dict = {
    0.0: "Unclassified",
    11.0: "Open Water",
    12.0: "Perennial Snow/Ice",
    21.0: "Developed, Open Space",
    22.0: "Developed, Low Intensity",
    23.0: "Developed, Medium Intensity",
    24.0: "Developed, High Intensity",
    31.0: "Barren Land",
    41.0: "Deciduous Forest",
    42.0: "Evergreen Forest",
    43.0: "Mixed Forest",
    52.0: "Shrub/Scrub",
    71.0: "Herbaceous",
    81.0: "Hay/Pasture",
    82.0: "Cultivated Crops",
    90.0: "Woody Wetlands",
    95.0: "Emergent Herbaceous Wetlands"
}
drought_dict = {
    -1: "No Drought",
    0.0: "D0",
    1.0: "D1",
    2.0: "D2",
    3.0: "D3",
    4.0: "D4"
}
def groundcover_value_replace(i):
    return groundcover_dict[i]
def drought_value_replace(i):
    return drought_dict[i]

groundcover_mapper = np.vectorize(groundcover_value_replace)
drought_mapper = np.vectorize(drought_value_replace)

this_year_df = pd.read_parquet("lat_long_height_agency.parquet")

for year in [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]:
    time_series_arrays = []
    print(f"-----------------------")
    for category in ["groundcover", "ppt", "tmp", "drought", "fire"]:
        print(f'ras_{category}{year}')
        thisRas = arcpy.Raster(r'C:\Users\augus\Documents\ArcGIS\Projects\PyROW\PyROW_CA.gdb' + f"\\ras_{category}{year}")
        this_flat_array = np.delete(arcpy.RasterToNumPyArray(thisRas, nodata_to_value=0).flatten(), drop_indices)

        if category == "drought":
            this_flat_array = drought_mapper(this_flat_array)
        elif category == "groundcover":
            this_flat_array = groundcover_mapper(this_flat_array)

        time_series_arrays.append(this_flat_array)

    time_arrays_for_df = {
        f"GROUNDCOVER_{year}": time_series_arrays[0],
        f"PRECIPITATION_{year}": time_series_arrays[1],
        f"TEMPERATURE_{year}": time_series_arrays[2],
        f"DROUGHT_{year}": time_series_arrays[3],
        f"FIRE_{year}": time_series_arrays[4]}

    time_df = pd.DataFrame(data=time_arrays_for_df)

    combo_df = pd.concat([this_year_df, time_df.reindex(this_year_df.index)], axis=1)

    combo_df.to_parquet(f"PyROW_data_year{year}.parquet", compression='None')

-----------------------
ras_groundcover2010
ras_ppt2010
ras_tmp2010
ras_drought2010
ras_fire2010
-----------------------
ras_groundcover2011
ras_ppt2011
ras_tmp2011
ras_drought2011
ras_fire2011
-----------------------
ras_groundcover2012
ras_ppt2012
ras_tmp2012
ras_drought2012
ras_fire2012
-----------------------
ras_groundcover2013
ras_ppt2013
ras_tmp2013
ras_drought2013
ras_fire2013
-----------------------
ras_groundcover2014
ras_ppt2014
ras_tmp2014
ras_drought2014
ras_fire2014
-----------------------
ras_groundcover2015
ras_ppt2015
ras_tmp2015
ras_drought2015
ras_fire2015
-----------------------
ras_groundcover2016
ras_ppt2016
ras_tmp2016
ras_drought2016
ras_fire2016
-----------------------
ras_groundcover2017
ras_ppt2017
ras_tmp2017
ras_drought2017
ras_fire2017
-----------------------
ras_groundcover2018
ras_ppt2018
ras_tmp2018
ras_drought2018
ras_fire2018
-----------------------
ras_groundcover2019
ras_ppt2019
ras_tmp2019
ras_drought2019
ras_fire2019
-----------------------
ras_gr